<a href="https://colab.research.google.com/github/sourcecode369/deep-nlp/blob/master/sequence%20classification/Sequence_Classification_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence Classification with Transformers

This colab notebook will guide you through using the Transformers library to obtain state-of-the-art results on the sequence classification task. It is attached to [the following tutorial](https://medium.com/@lysandrejik/using-tensorflow-2-for-state-of-the-art-natural-language-processing-102445cda54a).

We will be using two different models as a means of comparison: Google's BERT and Facebook's RoBERTa. Both have the same architecture but have had different pre-training approached.

## Installing required dependencies
In order to import the TensorFlow modules, we must make sure that TF2 is installed in the environment.

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
!pip install transformers

  Cloning https://github.com/huggingface/transformers (to revision tf_tutorial) to /tmp/pip-req-build-8bhed5ky
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-8bhed5ky
  Running command git checkout -b tf_tutorial --track origin/tf_tutorial
  Switched to a new branch 'tf_tutorial'
  Branch 'tf_tutorial' set up to track remote branch 'tf_tutorial' from 'origin'.
     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 45.6MB/s 
     |████████████████████████████████| 860kB 43.3MB/s 
  Created wheel for transformers: filename=transformers-2.1.1-cp36-none-any.whl size=311750 sha256=29188e48f76909baf0f06fc1d1911d8fac5cbbb96cc2a5b239b0dcb703fee687
  Stored in directory: /tmp/pip-ephem-wheel-cache-gp_v21do/wheels/9b/bd/d5/9c1efe0e778584b0006254ccec95a28b124b0a697fdc2dc410
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=2b90734be95ea4ca7c21fb55f891c44aee617

## Initializing the pre-trained models

Let's initialize the models with pre-trained weights. The list of pre-trained weights is available in [the official documentation](https://huggingface.co/transformers/pretrained_models.html). Downloading the weights may take a bit of time, but it only needs to be done once!

In [0]:
from transformers import (TFBertForSequenceClassification, 
                          BertTokenizer,
                          TFRobertaForSequenceClassification, 
                          RobertaTokenizer)

bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

roberta_model = TFRobertaForSequenceClassification.from_pretrained("roberta-base")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

100%|██████████| 456318/456318 [00:00<00:00, 513267.74B/s]


### Tokenization

BERT and RoBERTa are both Transformer models that have the same architecture. As such, they accept only a certain kind of inputs: vectors of integers, each value representing a token. Each string of text must first be converted to a list of indices to be fed to the model. The tokenizer takes care of that for us.

BERT and RoBERTa may have the same architecture, but they differ in tokenization. BERT uses a sub-word tokenization, whereas RoBERTa uses the same tokenization than GPT-2: byte-level byte-pair-encoding. Let's see what this means:

In [0]:
sequence = "Systolic arrays are cool. This 🐳 is cool too."

bert_tokenized_sequence = bert_tokenizer.tokenize(sequence)
roberta_tokenized_sequence = roberta_tokenizer.tokenize(sequence)

print("BERT:", bert_tokenized_sequence)
print("RoBERTa:", roberta_tokenized_sequence)

BERT: ['S', '##ys', '##to', '##lic', 'array', '##s', 'are', 'cool', '.', 'This', '[UNK]', 'is', 'cool', 'too', '.']
RoBERTa: ['Sy', 'st', 'olic', 'Ġarrays', 'Ġare', 'Ġcool', '.', 'ĠThis', 'ĠðŁ', 'Ĳ', '³', 'Ġis', 'Ġcool', 'Ġtoo', '.']


**BERT Tokenizer**

Here, the BERT tokenizer splits the string into multiple substrings. If the substrings are in its vocabulary, they will stay as is: this is the case for `array`,  `are` and  `cool`. However, if a resulting string is not in its vocabulary, it will be split again until every string is represented by its vocabulary. For example,  `Systolic` is split multiple times until every token is represented in the BERT vocabulary: it is split into four tokens.
The BERT tokenizer is lacking when it comes to complex characters spread over multiple bytes, as can be seen with emojis. In the sequence used, an emoji of a whale was added. As the BERT tokenizer cannot interpret this emoji on a byte-level, it replaces it by the unknown token [UNK].

**RoBERTa Tokenizer**

On the other hand, the RoBERTa tokenizer has a slightly different approach. Here too, the string is split into multiple substrings, which are themselves split into multiple substrings until every substring can be represented by the vocabulary. However, the RoBERTa tokenizer has a **byte-level approach**. This tokenizer can represent every sequence as a combination of bytes, which makes it shine in the case of complex characters spread over multiple bytes, as with the whale emoji. Instead of using the unknown token, this tokenizer can correctly encode the whale emoji as the combination of multiple bytes. This tokenizer therefore does not require an unknown token, as it can handle every byte separately.

## Getting State-of-the-Art results on sequence classification

In order to get State-of-the-Art results on this task, we will fine-tune our models on a given dataset. Fine-tuning a model means that we will slightly train it on top of an already trained checkpoint. The learning rate will be very low, as having it to high would result in catastrophic forgetting -> the model would forget what it had learned until now semantically and syntaxically.

We will follow the procedure detailed below:

    1) Get the dataset from `tensorflow_datasets`

    2) Pre-process this dataset so that it can be used by the model

    3) Set-up a training loop using Keras' fit API; train the model on the training data

    4) Evaluate the model on the testing data and compare to the actual results

### Getting the dataset

We will be using the Microsoft Research Paraphrase Corpus (MRPC) dataset, which is a sequence classification dataset. We get the train and validation data from the `tensorflow_datasets` package. These values are in the form of `tf.data.Dataset`, which is perfect for our use-case.

In [0]:
import tensorflow_datasets
data = tensorflow_datasets.load("glue/mrpc")

train_dataset = data["train"]
validation_dataset = data["validation"]

Let's output the value of the first item of the dataset to get an idea of the data we're dealing with.

In [0]:
example = list(train_dataset.__iter__())[0]
print('',
    'idx:      ', example['idx'],       '\n',
    'label:    ', example['label'],     '\n',
    'sentence1:', example['sentence1'], '\n',
    'sentence2:', example['sentence2'],
)

 idx:       tf.Tensor(201, shape=(), dtype=int32) 
 label:     tf.Tensor(1, shape=(), dtype=int64) 
 sentence1: tf.Tensor(b'Tibco has used the Rendezvous name since 1994 for several of its technology products , according to the Palo Alto , California company .', shape=(), dtype=string) 
 sentence2: tf.Tensor(b'Tibco has used the Rendezvous name since 1994 for several of its technology products , it said .', shape=(), dtype=string)


The values are in the form of dictionaries that are done as follow:

```py
example = {
    idx: number,
    label: number,
    sentence1: string,
    sentence2: string
}
```
The three fields that are of interest to us are the `label`, the `sentence1` and the `sentence2`. The label is equal to `1` when the two sentences are paraphrases of each other, and is equal to `0` otherwise.

We cannot pass this directly to the models as they cannot interpret the meaning of strings (list of characters), so we will see how to convert that example to features that our models can understand. Firstly, we must obtain the token ids that will be fed to the model from the two sequences. The two models (BERT, RoBERTa) have different encoding mechanisms concerning pair sequence encoding, using `sep` and `cls` tokens to specify the end of a sequence or the end of the two sequences. With `A` as the first sequence and `B` as the second, BERT encodes the sequence pair as follows:

`[CLS] A [SEP] B [SEP]`

while RoBERTa encodes the sequence pair differently:

`[CLS] A [SEP][SEP] B [SEP]`

Thankfully, our encode method can handle that on its own. Here are the two sentences we are currently dealing with:

In [0]:
seq0 = example['sentence1'].numpy().decode('utf-8')  # Obtain bytes from tensor and convert it to a string
seq1 = example['sentence2'].numpy().decode('utf-8')  # Obtain bytes from tensor and convert it to a string

print("First sequence:", seq0)
print("Second sequence:", seq1)

First sequence: Tibco has used the Rendezvous name since 1994 for several of its technology products , according to the Palo Alto , California company .
Second sequence: Tibco has used the Rendezvous name since 1994 for several of its technology products , it said .


## Encoding sequences

In order to encode the sequence to be understandable by the model, two different methods can come in handy.

### encode()

`encode` is a high-level method that returns the encoded sequence with the special tokens and truncated to a maximum length if need be. Here we identify the special CLS and SEP tokens of RoBERTa and BERT, and explicit them in the encoded sequence as to understand the difference in tokenization.

In [0]:
encoded_bert_sequence = bert_tokenizer.encode(seq0, seq1, add_special_tokens=True, max_length=128)
encoded_roberta_sequence = roberta_tokenizer.encode(seq0, seq1, add_special_tokens=True, max_length=128)

print("BERT tokenizer separator, cls token id:   ", bert_tokenizer.sep_token_id, bert_tokenizer.cls_token_id)
print("RoBERTa tokenizer separator, cls token id:", roberta_tokenizer.sep_token_id, roberta_tokenizer.cls_token_id)

bert_special_tokens = [bert_tokenizer.sep_token_id, bert_tokenizer.cls_token_id]
roberta_special_tokens = [roberta_tokenizer.sep_token_id, roberta_tokenizer.cls_token_id]

def print_in_red(string):
    print("\033[91m" + str(string) + "\033[0m", end=' ')

print("\nBERT tokenized sequence")
output = [print_in_red(tok) if tok in bert_special_tokens else print(tok, end=' ') for tok in encoded_bert_sequence]

print("\n\nRoBERTa tokenized sequence")
output = [print_in_red(tok) if tok in roberta_special_tokens else print(tok, end=' ') for tok in encoded_roberta_sequence]

BERT tokenizer separator, cls token id:    102 101
RoBERTa tokenizer separator, cls token id: 2 0

BERT tokenized sequence
101 157 13292 2528 1144 1215 1103 16513 15125 11944 1271 1290 1898 1111 1317 1104 1157 2815 2982 117 2452 1106 1103 19585 2858 17762 117 1756 1419 119 102 157 13292 2528 1144 1215 1103 16513 15125 11944 1271 1290 1898 1111 1317 1104 1157 2815 2982 117 1122 1163 119 102 

RoBERTa tokenized sequence
0 565 1452 876 34 341 5 29110 42057 766 187 8148 13 484 9 63 806 785 2156 309 7 5 21065 18402 2156 886 138 479 2 2 565 1452 876 34 341 5 29110 42057 766 187 8148 13 484 9 63 806 785 2156 24 26 479 2 

### encode_plus()

`encode_plus` is similar to `encode` but it returns additional information: the token type ids as well as several other features that we don't need to manage right now.

The token type ids are used by some models in the case of sequence classification. It is a mask indicating to the model which sequence a token is from. 

For example, let's say we have two sequences A and B with tokens `[a0, a1, a2, a3]` and `[b0, b1, b2, b3, b4]` respectively.

The BERT tokenizer would create a single sequence from those two lists of tokens that would look like the following:

<pre>
[tokens]         `[CLS] a0 a1 a2 a3 [SEP] b0 b1 b2 b3 b4 [SEP]`. 
[token type ids] `  0    0  0  0  0   0    1  1  1  1  1   1`
</pre>

Thanks to the token type ids, the model is aware of which token belongs to which sequence





We won't need to use encode_plus in this experiment as directly in the `Transformers` library exists a method to directly convert a dataset to features, and is agnostic to both the GLUE task and the specified tokenizer. This method makes use of `encode_plus` under the hood and is called `glue_convert_examples_to_features`:

In [0]:
from transformers import glue_convert_examples_to_features

bert_train_dataset = glue_convert_examples_to_features(train_dataset, bert_tokenizer, 128, 'mrpc')
bert_train_dataset = bert_train_dataset.shuffle(100).batch(32).repeat(2)

bert_validation_dataset = glue_convert_examples_to_features(validation_dataset, bert_tokenizer, 128, 'mrpc')
bert_validation_dataset = bert_validation_dataset.batch(64)

The two BERT datasets are now ready to be used: the training dataset is shuffled and batch, while the validation dataset is only batched.

RoBERTa requires a bit more of work as it does not use the `token_type_ids`, which we need to remove. We use the `tf.data.Dataset.map()` method for this.

In [0]:
def token_type_ids_removal(example, label):
    del example["token_type_ids"]
    return example, label

roberta_train_dataset = glue_convert_examples_to_features(train_dataset, roberta_tokenizer, 128, 'mrpc')
roberta_train_dataset = roberta_train_dataset.map(token_type_ids_removal)
roberta_train_dataset = roberta_train_dataset.shuffle(100).batch(32).repeat(2)

roberta_validation_dataset = glue_convert_examples_to_features(validation_dataset, roberta_tokenizer, 128, 'mrpc')
roberta_validation_dataset = roberta_validation_dataset.map(token_type_ids_removal)
roberta_validation_dataset = roberta_validation_dataset.batch(64)

### Defining the hyper-parameters

Before fine-tuning the model, we must define a few hyperparameters that will be used during the training such as the optimizer, the loss and the evaluation metric.

As an optimizer we'll be using Adam, which was the optimizer used during those models' pre-training. As a loss we'll be using the sparse categorical cross-entropy, and the sparse categorical accuracy as the evaluation metric.

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
roberta_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

### Training the model

The beauty of tensorflow/keras lies here: using keras' fit method to fine-tune the model with a single line of code

In [0]:
print("Fine-tuning BERT on MRPC")
bert_history = bert_model.fit(bert_train_dataset, epochs=3, validation_data=bert_validation_dataset)

print("\nFine-tuning RoBERTa on MRPC")
roberta_history = roberta_model.fit(roberta_train_dataset, epochs=3, validation_data=roberta_validation_dataset)

Fine-tuning BERT on MRPC
Epoch 1/3
230/230 [==============================] - 376s 2s/step - loss: 0.4896 - accuracy: 0.7601 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
230/230 [==============================] - 346s 2s/step - loss: 0.1541 - accuracy: 0.9434 - val_loss: 0.5791 - val_accuracy: 0.8456
Epoch 3/3
230/230 [==============================] - 347s 2s/step - loss: 0.0567 - accuracy: 0.9804 - val_loss: 0.5465 - val_accuracy: 0.8505

Fine-tuning RoBERTa on MRPC
Epoch 1/3


230/230 [==============================] - 377s 2s/step - loss: 0.6363 - accuracy: 0.6728 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
230/230 [==============================] - 349s 2s/step - loss: 0.6362 - accuracy: 0.6745 - val_loss: 0.6281 - val_accuracy: 0.6838
Epoch 3/3
230/230 [==============================] - 348s 2s/step - loss: 0.6351 - accuracy: 0.6745 - val_loss: 0.6280 - val_accuracy: 0.6838


### Keras's simplicity doesn't end here

Evaluating a model is as simple as it is to train it - using the evaluate method

In [0]:
print("Evaluating the BERT model")
bert_model.evaluate(bert_validation_dataset)

print("Evaluating the RoBERTa model")
roberta_model.evaluate(roberta_validation_dataset)

## Results

The results we obtain for BERT are similar to the paper's original results, which were computed using the official GLUE evaluation server. The accuracy obtained for RoBERTa is slightly less than in the paper, which is probably due to the initialisation done: in the paper, the fine-tuning on the MRPC task was done from the MNLI checkpoint rather than from the base checkpoint.